In [1]:
import tensorflow as tf

2024-01-23 21:40:08.791675: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 21:40:08.812082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 21:40:08.812103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 21:40:08.812680: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 21:40:08.816062: I tensorflow/core/platform/cpu_feature_guar

In [6]:
# Load and preprocess the MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [7]:
# Define the LeNet model in TensorFlow
model_tf = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='sigmoid'),
    tf.keras.layers.Dense(84, activation='sigmoid'),
    tf.keras.layers.Dense(10)  # Assuming 10 classes
])

# Compile the model
model_tf.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

# Train the model
history = model_tf.fit(train_images, train_labels, epochs=3, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/3


2024-01-23 21:45:14.818795: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-23 21:45:14.868079: I external/local_xla/xla/service/service.cc:168] XLA service 0x5600ec013ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-23 21:45:14.868098: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-01-23 21:45:14.872936: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706017514.918089   71597 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1688/1688 [==============================] - 4s 2ms/step - loss: 0.8231 - accuracy: 0.7373 - val_loss: 0.2165 - val_accuracy: 0.9373
Epoch 2/3
1688/1688 [==============================] - 3s 2ms/step - loss: 0.2051 - accuracy: 0.9391 - val_loss: 0.1294 - val_accuracy: 0.9618
Epoch 3/3
1688/1688 [==============================] - 3s 2ms/step - loss: 0.1313 - accuracy: 0.9601 - val_loss: 0.0896 - val_accuracy: 0.9747
313/313 - 0s - loss: 0.1025 - accuracy: 0.9677 - 208ms/epoch - 665us/step

Test accuracy: 0.9677000045776367


## Layer 1

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)

    def forward(self, x):
        return torch.sigmoid(self.conv1(x))

model_pt_basic = BasicLeNetPT()

model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1))
])

In [10]:
import numpy as np
# Transfer weights for TensorFlow
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for PyTorch
with torch.no_grad():
    model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
    model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))


In [18]:
# Create a controlled input (e.g., an array of ones)
controlled_input = np.ones((1, 28, 28, 1), dtype=np.float32)  # For TensorFlow
controlled_input_pt = torch.from_numpy(controlled_input).permute(0, 3, 1, 2)  # For PyTorch


In [19]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)

# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().permute(3,2,0,1).numpy())


1/1 [==============================] - 0s 14ms/step
TensorFlow Basic Model Output: [[[[9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]
   [9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]
   [9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]
   ...
   [9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]
   [9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]
   [9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]]

  [[9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]
   [9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]
   [9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    7.6754862e-01 3.5114321e-05]
   ...
   [9.9660897e-01 2.1650463e-01 4.8949671e-01 8.8328234e-04
    

## Layer 2

In [36]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x = torch.sigmoid(self.conv2(x))
        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid')
])

In [42]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[1].set_weights([weights, biases])

# Transfer weights for the first Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))


In [45]:
# Create a controlled input (e.g., an array of ones)
controlled_input = np.ones((1, 28, 28, 1), dtype=np.float32)  # For TensorFlow
controlled_input_pt = torch.from_numpy(controlled_input).permute(0, 3, 1, 2)  # For PyTorch


In [46]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().permute(3,2,0,1).numpy())


1/1 [==============================] - 0s 9ms/step
TensorFlow Basic Model Output: [[[[0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   ...
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]]

  [[0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   ...
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   [0.84262574 0.4728635  0.9888236  ... 0.60340494 0.36525252
    0.15889616]
   [0.84262574 0.4728635  0.98882

## AvgPool

In [52]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2))
])

In [53]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[2].set_weights([weights, biases])

# Transfer weights for the first Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))


In [54]:
# Create a controlled input (e.g., an array of ones)
controlled_input = np.ones((1, 28, 28, 1), dtype=np.float32)  # For TensorFlow
controlled_input_pt = torch.from_numpy(controlled_input).permute(0, 3, 1, 2)  # For PyTorch


In [55]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().permute(3,2,0,1).numpy())


1/1 [==============================] - 0s 34ms/step
TensorFlow Basic Model Output: [[[[8.4262574e-01 4.7286350e-01 9.8882359e-01 7.0327699e-01
    1.5978925e-01 3.1689417e-01 2.0221945e-02 6.9187993e-01
    6.5178031e-01 8.5156304e-01 9.5023102e-01 6.4571332e-03
    7.1215414e-04 6.0340494e-01 3.6525252e-01 1.5889616e-01]
   [8.4262574e-01 4.7286350e-01 9.8882359e-01 7.0327699e-01
    1.5978925e-01 3.1689417e-01 2.0221945e-02 6.9187993e-01
    6.5178031e-01 8.5156304e-01 9.5023102e-01 6.4571332e-03
    7.1215414e-04 6.0340494e-01 3.6525252e-01 1.5889616e-01]
   [8.4262574e-01 4.7286350e-01 9.8882359e-01 7.0327699e-01
    1.5978925e-01 3.1689417e-01 2.0221945e-02 6.9187993e-01
    6.5178031e-01 8.5156304e-01 9.5023102e-01 6.4571332e-03
    7.1215414e-04 6.0340494e-01 3.6525252e-01 1.5889616e-01]
   [8.4262574e-01 4.7286350e-01 9.8882359e-01 7.0327699e-01
    1.5978925e-01 3.1689417e-01 2.0221945e-02 6.9187993e-01
    6.5178031e-01 8.5156304e-01 9.5023102e-01 6.4571332e-03
    7.1215414e